# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откроем файл с данными и изучим общую информацию

In [1]:
import pandas as pd
import numpy as np

from pymystem3 import Mystem
m = Mystem()

from collections import Counter

import warnings
warnings.simplefilter("ignore")


data = pd.read_csv('/datasets/data.csv')
data.info()
data.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

В каждой строке таблицы имеется информация о клиенте банка - данные о возрасте, образовании, гендерной принадлежности, семейном положении, количестве детей, его трудоустройстве (должность, сколько дней проработал, какой доход) и имел ли клиент задолженность по возврату кредита.

На что необходимо обратить внимание:
* В столбцах 'days_employed' и 'total_income' есть пропущенные значения.
* В столбце 'days_employed' много отрицательных значений, тип данных - float и присутствуют значения, превышающие логические показатели.
* В столбце 'children' также есть отрицательные значения.
* Данные столбца 'education' записаны разным регистром.
* Есть нулевые значения в столбце 'dob_years'.


## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
print(data.isna().sum())

# пропуски income_type и days_employed
# замена пропусков в столбце с доходами на медианные значения
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'))

# замена пропусков в столбце со стажем на средние значения
data['days_employed'] = data['days_employed'].fillna(data.groupby('dob_years')['days_employed'].transform('mean'))

# проверка замен
print(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


In [3]:
# отрицательные days_employed

# преобразуем отрицательные значения в положительные
data['days_employed'] = data['days_employed'].map(abs, ['days_employed'])

# пересчитвыем превышенные значения

for index in range(len(data)):
    if data['days_employed'][index] > 16000:
        data['days_employed'][index] = data['days_employed'][index] / 24
        
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
# регистр education и family_status

data['education'] = data['education'].str.lower()
print(data.groupby('education')['education'].count())

data['family_status'] = data['family_status'].str.lower()
print(data.groupby('family_status')['family_status'].count())

education
высшее                  5260
начальное                282
неоконченное высшее      744
среднее                15233
ученая степень             6
Name: education, dtype: int64
family_status
в разводе                 1195
вдовец / вдова             960
гражданский брак          4177
женат / замужем          12380
не женат / не замужем     2813
Name: family_status, dtype: int64


In [5]:
# отрицательные children
print(data.groupby('children')['children'].count())

# преобразуем отрицательные значения в положительные
data['children'] = data['children'].map(abs, ['children'])

display(data.tail(8))

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21517,0,404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,15583.154618,59,среднее,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,14330.725172,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


**Вывод**

Пропуски NaN ищем с помощью функции isna() и суммируем все функцией sum().
NaN находятся в столбцах 'total_income' и 'days_employed'. По количеству пропущенные значения совпадают.
Возможно, при заполнении данных, не были указаны стаж и доход. Или же доход расчитывался на основании стажа, и при отсутвующем стаже в  доход также попали пропуски. Это может объяснить, почему строки по эти двум столбца одинаково пропущены.

Пропущенные значения в столбце 'total_income' заменяем на медиану, расчитанную для каждой группы в стобце 'income_type', а в стлбце 'days_employed' на среднее значение по возрасту.

Для преобразования отрицательных значений применяем встроенную функцию abs() к столбцу 'days_employed'.
Касательно больших значений в столбце 'days_employed': предположим, что при заполнении некоторые данные перевели в часы. В таком случае возьмем в среднем стаж работы около 16 000 дней. Циклом перебираем значения в столбце 'days_employed' больше 16000 и меняем их на примерный стаж согласно возрасту.

Данные в столбцах выглядят более логичными.

Также в столбцах 'education' и 'family_status' используется разный регистр. Приводим данные к общему регистру методом lower().

Отрицательные значения в столбце 'children'. Таких значений 47, скорей всего какая-то техническая ошибка. Также убираем минуса применив встроенную функцию abs(). Вот с количеством 20 детей немного нпонятно, либо 0 лишний либо реальное количество умножили на какой-то коэффициент.  Пока оставим это значение без изменений, если далее в расчетах будем использовать условие 'более 2 детей' например.

### Замена типа данных

In [6]:
# замена типа у 'days_employed'
data['days_employed'] = data['days_employed'].astype(int)
data['total_income'] = data['total_income'].astype(int)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
None


**Вывод**

В таблице только у одного столбца тип данных, который бросается в глаза и портит весь вид. Меняем тип float на int с помощью метода astype(), так как этот метод применим к Series.

### Обработка дубликатов

In [7]:
# поиск дубликатов 'purpose'
print(data.duplicated().value_counts())
print(data['purpose'].value_counts())


False    21454
True        71
dtype: int64
свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля         

**Вывод**

С помощью метода value_counts() проверяем столбец 'purpose' на уникальные повторящиеся значения. Данный столбец явно имеет дубликаты, которые необходимо объединить в несколько общих групп.
Такие дубликаты появились, скорей всего, потому что, данные заполнялись клиентом, либо несколькими менеджерами банка, которые привыкли к использованию своих обозначений целей.

### Лемматизация

In [8]:
# проверим какие общие группы есть в 'purpose'
lem_data = list(data['purpose'])
lem_data = ' '.join(lem_data)
lemmas = m.lemmatize(lem_data)
print(Counter(lemmas))

Counter({' ': 55201, 'недвижимость': 6367, 'покупка': 5912, 'жилье': 4473, 'автомобиль': 4315, 'образование': 4022, 'с': 2924, 'операция': 2610, 'свадьба': 2348, 'свой': 2235, 'на': 2233, 'строительство': 1881, 'высокий': 1375, 'получение': 1316, 'коммерческий': 1315, 'для': 1294, 'жилой': 1233, 'сделка': 944, 'дополнительный': 909, 'заниматься': 908, 'подержать': 858, 'проведение': 777, 'сыграть': 774, 'сдача': 653, 'семья': 641, 'собственный': 635, 'со': 630, 'ремонт': 612, 'приобретение': 462, 'профильный': 436, 'подержанный': 110, '\n': 1})


**Вывод**

Для того чтобы лемматизировать данные, необходимо преобразовать все цели из столбца в строку - применим функцию list(), которая возвращает список значений. Далее методом join() с разделителем 'пробел' превращаем наш список в строку. 
После чего можно воспользоваться лемматизацией и подсчитать леммы. Подсчет  произвела функцией Counter(), изначально импортировав из модуля collections.

Применив лемматизацию к столбцу 'purpose' можно выявить 4 основные цели кредитоваания: 
* недвижимость
* автомобиль
* образование
* свадьба


### Категоризация данных

In [9]:
def purpose_grouped(row):
    purpose = row['purpose']
    
    if 'свадьб' in purpose:
        return 'свадьба'
    
    if 'недвиж' in purpose:
        return 'недвижимость'
    
    if 'жиль' in purpose:
        return 'недвижимость'
    
    if 'образов' in purpose:
        return 'образование'
    
    if 'автомоб' in purpose:
        return 'автомобиль'
    
data['purpose'] = data.apply(purpose_grouped, axis=1)
display(data.head())


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба


**Вывод**

Пишем функцию для замены данных в столбце 'purpose', к столбцу применяем функцию методом apply(). Данные категоризированы по целям.

## Шаг 3. Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [10]:
# добавим столбец подкатегорию для children

data['children_category'] = np.where(data.children > 0, 'есть дети', 'нет детей')

data_pivot_children = data.pivot_table(index=['children_category'], values='debt', aggfunc='mean')
display(data_pivot_children)

,debt
children_category,
есть дети,0.091920
нет детей,0.075129


**Вывод**

Выделили клиентов по двум группам - 'есть дети' и 'нет детей'. Посчитали среднее значение задолженностей по каждой группе.
Согласно сводной таблице, наличие детей влияет на погашение кредита в срок. Доля задолженностей у клиентов с детьми (9,2%) выше доли клиентов, у которых нет детей (7,5%).

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [11]:
data_pivot_status = data.pivot_table(index=['family_status_id', 'family_status'], values='debt', aggfunc='mean')

display(data_pivot_status) 

,,debt
family_status_id,family_status,
0,женат / замужем,0.075202
1,гражданский брак,0.092890
2,вдовец / вдова,0.065625
3,в разводе,0.071130
4,не женат / не замужем,0.097405


**Вывод**

Если соотнести среднее занчение долгов с семейным положением, видно, что люди, не состоящие и никогда не состоявшие прежде в браке (9,3% - 9,7%) имеют задолженность чаще чем женатые / замужние (7,5%) или разведенные (7,1%). И отдельно можно выделить вдов / вдовцов, у них этот показатель ниже всех (6,6%).

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [12]:
# уровни дохода разделим на 3 основыние группы

high_income = data['total_income'].max()
avg_income = data['total_income'].median()
low_income = data['total_income'].min()

print(high_income)
print(avg_income)
print(low_income)

# добавим столбец 'income_category' с помощью функции

def function_income (row):
    income = row['total_income']
    
    if income <= 100000:
        return 'низкий уровень'
    
    if income <= 400000:
        return 'средний уровень'
    
    if income > 400000:
        return 'высокий уровень'
    
data['income_level'] = data.apply(function_income, axis=1)

data_pivot_income = data.pivot_table(index=['income_level'], values='debt', aggfunc=['count', 'sum', 'mean'])

display(data_pivot_income)

2265604
142594.0
20667


,count,sum,mean
,debt,debt,debt
income_level,,,
высокий уровень,529,31,0.058601
низкий уровень,4463,354,0.079319
средний уровень,16533,1356,0.082018


**Вывод**

На все 21 тысячу строк имеется разный доход. Функциями max(), median(), min() выявим три основые точки. Исходя из полученных данных можно сделать вывод, что максимальное и минимальное значения могут быть единичными в таком большом датасете. Предположим, что низкий уровень дохода это суммы меньше 100 тысяч, средний - меньше 400, а высокий все суммы больше 400 тысяч. Добавим еще один столбец для группировки дохода.
Формируя сводную таблицу, добавим функции 'count' для подсчета количества по группам и 'sum' для подсчета задолженностей на каждую группу, чтобы проанализировать насколько реально выглядят данные.

На основании сводной можно сделать вывод, что как таковой зависимости между уровнем дохода и возвратом кредита в срок нет, т.к. клиенты банка со средним уровнем дохода чаще бывают должниками, чем клиенты с низким уровнем. 

- Как разные цели кредита влияют на его возврат в срок?

In [13]:
data_pivot_purpose = data.pivot_table(index=['purpose'], values='debt', aggfunc=['count', 'sum', 'mean'])
data_pivot_status = data.pivot_table(index=['purpose', 'family_status'], values='debt', aggfunc=['count', 'sum', 'mean'])

display(data_pivot_purpose)
display(data_pivot_status)

,count,sum,mean
,debt,debt,debt
purpose,,,
автомобиль,4315,403,0.093395
недвижимость,10840,782,0.072140
образование,4022,370,0.091994
свадьба,2348,186,0.079216


count  sum      mean
                                    debt debt      debt
purpose      family_status                             
автомобиль   в разводе               281   21  0.074733
             вдовец / вдова          218   20  0.091743
             гражданский брак        434   51  0.117512
             женат / замужем        2744  229  0.083455
             не женат / не замужем   638   82  0.128527
недвижимость в разводе               676   47  0.069527
             вдовец / вдова          543   28  0.051565
             гражданский брак        991   91  0.091826
             женат / замужем        7033  486  0.069103
             не женат / не замужем  1597  130  0.081403
образование  в разводе               238   17  0.071429
             вдовец / вдова          199   15  0.075377
             гражданский брак        404   60  0.148515
             женат / замужем        2603  216  0.082981
             не женат / не замужем   578   62  0.107266
свадьба      гражданский брак       2348  186  0.079216

In [14]:
# дополнительно
# проверим какие типы занятости имеют 0 в возрасте и заменим на медианные значения

data_pivot_age = data.pivot_table(index=['dob_years', 'income_type'], values='debt', aggfunc='count')
#display(data_pivot_age)

# медианы для каждого типа занятости
data_grouped_age = data.groupby('income_type')['dob_years'].median()
display(data_grouped_age)

civil_servant = 40
companion = 39
retired = 60
employee = 39

for index in range(len(data)):
    if data['income_type'][index] == 'госслужащий' and data['dob_years'][index] == 0:
        data['dob_years'][index] = civil_servant
    if data['income_type'][index] == 'компаньон' and data['dob_years'][index] == 0:
        data['dob_years'][index] = companion
    if data['income_type'][index] == 'пенсионер' and data['dob_years'][index] == 0:
        data['dob_years'][index] = retired
    if data['income_type'][index] == 'сотрудник' and data['dob_years'][index] == 0:
        data['dob_years'][index] = employee

display(data_pivot_age)

income_type
безработный        38.0
в декрете          39.0
госслужащий        40.0
компаньон          39.0
пенсионер          60.0
предприниматель    42.5
сотрудник          39.0
студент            22.0
Name: dob_years, dtype: float64

debt
dob_years income_type      
0         госслужащий     6
          компаньон      20
          пенсионер      20
          сотрудник      55
19        госслужащий     1
...                     ...
73        пенсионер       7
74        компаньон       1
          пенсионер       4
          сотрудник       1
75        госслужащий     1

[221 rows x 1 columns]

In [15]:
# дополнительно по собственному желанию
# распределим на группы по возрасту

def function_age (row):
    age = row['dob_years']
    
    if age <= 35:
        return 'группа до 35'
    
    if age > 35 and age <= 45:
        return 'группа 36-45'
    
    if age > 45 and age <= 60:
        return 'группа 46-60'
    
    if age > 61:
        return 'группа 60+'
    
data['age_groups'] = data.apply(function_age, axis=1)



data_pivot_age_children = data.pivot_table(index=['children_category', 'age_groups'], values='debt', aggfunc=['count', 'sum', 'mean'])



display(data_pivot_age_children)

count  sum      mean
                                debt debt      debt
children_category age_groups                       
есть дети         группа 36-45  2848  243  0.085323
                  группа 46-60  1111   82  0.073807
                  группа 60+     111    5  0.045045
                  группа до 35  3273  347  0.106019
нет детей         группа 36-45  2866  231  0.080600
                  группа 46-60  5966  386  0.064700
                  группа 60+    1674   86  0.051374
                  группа до 35  3321  351  0.105691

С помощью сводной талицы проверяем в каких категориях, относящихся к столбцу 'income_type', присутствуют значения возраста равные 0. Таких категорий 4:
* госслужащий
* компаньон
* пенсионер
* сотрудник

Дале заменим нули на медианное значение по типу занятости с помощью цикла

Далее возьмем возрастные категории и распределим их по группам:
* до 35
* 36-45
* 46-60
* от 61 и выше

Построим сводную таблицу и рассмотрим такой вопрос "Как влияет наличие/отсутвисе детей и возраст на возврат кредита в срок?"

Клиенты, у которых есть дети в возрастной категории 36-45 имеют задолженность по кредитам значительно чаще (9,3%) чем клиенты в возрасте 46-60 также сдетьми (5,6%). И показатели задолженностей по тем же возрастным категориям, но у клиентов, у которых нет детей примерно такие же: возраст 36-45 - 8,1%, возраст 46-60 - 5,5%. 

Согласно полученным данным, можно сделать вывод, что задолженности по кредитам у клиентов с детьми больше чем у клиентов без детей. Здесь мы еще раз подтверждаем ответ на первый вопрос нашего анализа. Касательно возраста, чаще всего должникам по кредитам являются клиенты возрастной категории 36-45.
Группу людей до 35 можно не рассматривать либо обединить со второй группой, но они не сильно повлияют на анализ, так как по нашей таблице таких людей всего 3 человека.

**Вывод**

В основном возврат кредита в срок не зависит от целей. Здесь лучше добавить к анализу еще какие-то данные. 

Задолженности по целям авто и образование немного выше, чем по целям недвижимость и свадьба.
Такое проиходит, скорей всего, потому что, срок кредитования на недвижимость значительно выше чем на другие три группы, соответсвенно процентная ставка будет ниже, и как правило, недвижимость берут пары или супруги.

Выведем дополнительно сводную по семейному положению и цели кредитования.
Большая часть кредитов на недвижимость приходится на тех, кто в браке.

Автомобили и образование имеют задолженностей больше, так как срок кредитования меньше и ставка выше. И к тому же на образование кредиты берут в основном студенты, а им сложнее зарабатывать без образования.

Свадьбы. На свадьбы берут кредиты только пары в гражданском браке, это опять же пары, которые скорей всего уже делят семейный бюджет. Плюс сумма кредитования на свадьбу не большая.

## Шаг 4. Общий вывод

В процессе анализа мы проверили 4 вопроса:

1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
Полученные нами данные показали, что наличие детей влияет на погашение кредита в срок.

2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
Семейное положение влияет на погашение - одинокие или люди не в браке имеют задолженность чаще.

3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Уровень дохода почти не влияет на погашение в срок - низкий и средний уровень дохода клиента показал почи одинаковый процент.

4. Как разные цели кредита влияют на его возврат в срок?
Здесь можно сказать, что от целей зависит, как клиент справляется с задолженностью, но это слишком общие данные. Лучше запросить у банка дополнительную информацию например по процентным ставкам или срокам кредитования.
